<a href="https://colab.research.google.com/github/kimjy-par/ai_data_analyzie_course/blob/main/day4_autonomous_data_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YouTube 채널 분석 실습 개요

* * *

본 실습은 YouTube Data API와 Google Gemini API를 활용하여 특정 YouTube 채널의 영상 및 댓글 데이터를 수집하고 분석하는 과정을 다룹니다. 이전 실습에서 수동으로 진행했던 각 단계를, **미리 작성된 함수를 활용하여 간단하게 자동으로 실행하고 그 결과를 확인하는 데 중점을 둡니다.** 수집된 데이터를 바탕으로 AI 모델을 통해 심층적인 분석 리포트를 생성하고, 이를 통해 콘텐츠 기획 및 채널 운영 전략에 대한 실질적인 인사이트를 얻는 것을 목표로 합니다.

# 실습 목표:

* * *

1.  **미리 작성된 함수를 활용하여 각 단계 별로 처리된 결과를 자동으로 확인하고 전체 분석 파이프라인을 실행할 수 있다.**
2.  **수집 설정 (영상 개수, 댓글 개수 등) 및 AI 프롬프트를 수정하여 분석 결과가 어떻게 달라지는지 확인하고 결과를 분석할 수 있다.**


# 실습 전 준비사항:

* * *

(이전 시간과 동일)
1. YouTube Data API 키
2. Google Gemini API 키
3. 분석하고 싶은 유튜브 채널

# 실습 절차
* * *

1.  **필요 라이브러리 설치 및 환경 설정**: YouTube API 및 Gemini API 사용을 위한 라이브러리를 설치하고 API 키를 설정합니다. Google Drive 연결을 통해 분석 결과를 저장할 환경을 준비합니다.
2.  **YouTube 데이터 수집**: 특정 채널의 동영상 목록을 가져오고, 각 영상에 대한 댓글을 수집합니다. 수집 범위 (영상 개수, 댓글 개수 등)를 설정하여 원하는 데이터를 확보합니다.
3.  **데이터 전처리 및 저장**: 수집된 영상 및 댓글 데이터를 분석 가능한 형태로 가공하고, 추후 활용을 위해 파일로 저장합니다.
4.  **AI 분석 리포트 생성**: 전처리된 데이터를 바탕으로 Gemini AI 모델에 분석을 요청하고, AI가 생성한 분석 리포트를 확인합니다.

* * *

**참고사항:**

*   실습 중 API 할당량에 유의하여 진행합니다.

In [1]:
# 필요한 라이브러리를 모두 설치합니다 (한번만 실행)
!pip install yt-dlp google-api-python-client google-genai wordcloud

# 필요한 모든 라이브러리를 한번에 불러옵니다
import os                      # 파일과 폴더 관리용
import json                    # JSON 데이터 처리용
import time                    # 시간 지연용
import pandas as pd            # 데이터 분석용 (엑셀과 비슷)
import matplotlib.pyplot as plt # 그래프 그리기용
import numpy as np             # 수학 계산용
from collections import Counter # 빈도 계산용
import re                      # 텍스트 처리용
from datetime import datetime, timedelta  # 날짜 계산용
import subprocess              # 외부 프로그램 실행용


# Google Drive를 연결합니다 (결과 저장용)
from google.colab import drive
drive.mount('/content/drive')


# YouTube 데이터를 가져오는 라이브러리
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# AI 분석용 라이브러리
from google import genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.2 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
# 환경 설정
YOUTUBE_API_KEY = "유튜브 API KEY"  # YouTube Data API 키
GEMINI_API_KEY = "GEMINI API KEY"    # Gemini AI 키

In [11]:
MAX_VIDEOS = 10             # 분석할 영상 개수 (10개)
MAX_RAW_VIDEOS = 500       # 인기순 혹은 최신순 정렬을 위해 가져올 비디오 개수
MAX_COMMENTS_PER_VIDEO = 20 # 영상당 댓글 개수 (20개)

# 결과 저장할 폴더 설정
SAVE_PATH = "/content/drive/MyDrive/youtube_analysis/"
os.makedirs(SAVE_PATH, exist_ok=True)  # 폴더가 없으면 자동 생성

print("설정 완료!")
print(f"저장 위치: {SAVE_PATH}")


# =================================================================
# API 연결 설정
# =================================================================

# YouTube Data API 연결
try:
    youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)
    print("YouTube API 연결 성공!")
except Exception as e:
    print(f"YouTube API 연결 실패: {e}")
    print("API 키를 확인해주세요")

# Gemini AI 연결
try:
    os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
    genai_client = genai.Client()
    print("Gemini AI 연결 성공!")
except Exception as e:
    print(f"Gemini AI 연결 실패: {e}")
    print("API 키를 확인해주세요")



def get_channel_id_by_handle(handle):
    """채널 핸들로 실제 채널 ID를 찾는 함수"""
    try:
        handle = handle.replace('@', '')
        request = youtube.search().list(
            part='snippet',
            q=handle,
            type='channel',
            maxResults=1
        )
        response = request.execute()
        if response['items']:
            return response['items'][0]['snippet']['channelId']
        return None
    except Exception as e:
        print(f"채널 ID 찾기 에러: {e}")
        return None

def get_all_channel_videos(youtube, channel_id, limit=MAX_RAW_VIDEOS):
    """
    지정된 채널의 동영상 상세 정보를 지정된 최대 개수(limit)만큼만 가져옵니다.

    Args:
        youtube: YouTube API 서비스 객체
        channel_id (str): 조회할 채널의 ID
        limit (int): 가져올 최대 동영상 개수

    Returns:
        list: 채널의 동영상 정보가 담긴 리스트
    """
    try:
        # (이전과 동일) 채널의 '업로드' 재생목록 ID 가져오기
        channel_response = youtube.channels().list(part='contentDetails', id=channel_id).execute()
        uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

        video_ids = []
        next_page_token = None

        # while 루프 시작
        while True:
            playlist_response = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=uploads_playlist_id,
                maxResults=50,
                pageToken=next_page_token
            ).execute()

            video_ids.extend([item['contentDetails']['videoId'] for item in playlist_response['items']])

            # --- ✨ 핵심 수정 부분 ---
            # 1. 현재까지 가져온 동영상 개수가 limit을 넘었는지 확인
            if len(video_ids) >= limit:
                break # 넘었다면 루프 중단

            next_page_token = playlist_response.get('nextPageToken')

            # 2. 다음 페이지가 없어도 루프 중단
            if not next_page_token:
                break

        # --- ✨ 가져온 ID 개수에 맞춰 자르기 ---
        # limit을 초과해서 가져왔을 수 있으므로 정확히 limit 개수만큼만 잘라냅니다.
        limited_video_ids = video_ids[:limit]

        all_videos = []
        for i in range(0, len(limited_video_ids), 50):
            videos_response = youtube.videos().list(
                part='snippet,statistics,contentDetails',
                id=','.join(limited_video_ids[i:i+50])
            ).execute()
            all_videos.extend(videos_response['items'])

        return all_videos

    except Exception as e:
        print(f"데이터 수집 중 에러 발생: {e}")
        return []

def analyze_channel_videos(videos, sort_by='popular', start_date=None, end_date=None, limit=MAX_VIDEOS):
    """
    가져온 동영상 리스트를 주어진 조건에 따라 필터링하고 정렬합니다.

    Args:
        videos (list): 분석할 전체 동영상 리스트
        sort_by (str): 'popular' (인기순), 'latest' (최신순) 중 하나
        start_date (str): 'YYYY-MM-DD' 형식의 시작 날짜
        end_date (str): 'YYYY-MM-DD' 형식의 종료 날짜

    Returns:
        list: 조건에 맞게 처리된 동영상 리스트
    """
    processed_videos = videos

    # 1. 날짜 필터링
    if start_date and end_date:
        start_dt = datetime.fromisoformat(start_date + 'T00:00:00Z')
        end_dt = datetime.fromisoformat(end_date + 'T23:59:59Z')
        processed_videos = [
            v for v in processed_videos
            if start_dt <= datetime.fromisoformat(v['snippet']['publishedAt'].replace('Z', '+00:00')) <= end_dt
        ]

    # 2. 정렬
    if sort_by == 'popular':
        processed_videos = sorted(
            processed_videos,
            key=lambda v: int(v['statistics'].get('viewCount', 0)),
            reverse=True
        )
    elif sort_by == 'latest':
        processed_videos = sorted(
            processed_videos,
            key=lambda v: v['snippet']['publishedAt'],
            reverse=True
        )

    return processed_videos[:limit]

def convert_dataframe_from_video_data(videos):
  videos_data = []
  for video in videos:
    video_data = {
        "video_id": video.get("id"),
        "title": video.get("snippet").get("title"),
        "channel_title": video.get("snippet").get("channelTitle"),
        "upload_date": video.get("snippet").get("publishedAt"),
        "view_count": video.get("statistics").get("viewCount"),
        "like_count": video.get("statistics").get("likeCount"),
        "comment_count": video.get("statistics").get("commentCount"),
        "duration": video.get("contentDetails").get("duration"),
        "description": video.get("snippet").get("description"),
        "collected_at": datetime.now().isoformat()
    }
    videos_data.append(video_data)

  videos_df = pd.DataFrame(videos_data)
  return videos_df

def collect_video_comments(videos_df, youtube=youtube, max_comments_per_video=MAX_COMMENTS_PER_VIDEO):
    """
    주어진 동영상 목록(DataFrame)에 대해 댓글을 수집하여 DataFrame으로 반환합니다.

    Args:
        youtube: YouTube API 서비스 객체
        videos_df (pd.DataFrame): 'video_id', 'title' 컬럼을 포함한 데이터프레임
        max_comments_per_video (int): 영상당 수집할 최대 댓글 수

    Returns:
        pd.DataFrame: 수집된 모든 댓글 정보가 담긴 데이터프레임
    """
    if videos_df.empty:
        print("분석할 영상이 없어 댓글 수집을 건너뜁니다.")
        return pd.DataFrame()

    all_comments = []
    print("\n=================================================================")
    print("영상 댓글 수집 시작")
    print("=================================================================")

    for idx, video in videos_df.iterrows():
        video_id = video['video_id']
        video_title = video['title']
        print(f"\n[{idx+1}/{len(videos_df)}] '{video_title[:40]}...' 댓글 수집 중...")

        try:
            comments_request = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=max_comments_per_video,
                order='relevance'  # 인기순 (또는 'time'으로 최신순)
            )
            comments_response = comments_request.execute()

            video_comments = []
            for item in comments_response['items']:
                top_comment = item['snippet']['topLevelComment']['snippet']
                comment_data = {
                    'video_id': video_id,
                    'video_title': video_title,
                    'comment_id': item['snippet']['topLevelComment']['id'],
                    'text': top_comment['textDisplay'],
                    'author': top_comment['authorDisplayName'],
                    'like_count': top_comment.get('likeCount', 0),
                    'published_at': top_comment['publishedAt'],
                    'reply_count': item['snippet'].get('totalReplyCount', 0),
                    'collected_at': datetime.now().isoformat()
                }
                video_comments.append(comment_data)

            all_comments.extend(video_comments)
            print(f"  ✅ {len(video_comments)}개 댓글 수집 완료")
            time.sleep(0.5)

        except HttpError as e:
            error_message = str(e)
            if 'commentsDisabled' in error_message:
                print("  ⚠️ 댓글이 비활성화된 영상입니다.")
            elif 'quotaExceeded' in error_message:
                print("  🛑 API 사용량 초과! 댓글 수집을 중단합니다.")
                break
            else:
                print(f"  ❌ API 오류: {error_message}")
        except Exception as e:
            print(f"  ❌ 댓글 수집 중 알 수 없는 오류: {e}")

    print(f"\n댓글 수집 완료! 총 {len(all_comments)}개 댓글 수집")
    return pd.DataFrame(all_comments)

def preprocess_comments(comments_df):
    """
    댓글 데이터프레임을 받아 기본적인 전처리를 수행합니다.

    Args:
        comments_df (pd.DataFrame): 원본 댓글 데이터프레임

    Returns:
        pd.DataFrame: 필요한 컬럼만 선택하고 텍스트의 양끝 공백을 제거한 데이터프레임
    """
    if comments_df.empty:
        print("댓글 데이터가 없어 데이터 정리를 건너뜁니다.")
        return pd.DataFrame()

    # 필요한 컬럼만 선택하여 새로운 데이터프레임 생성
    processed_df = comments_df[[
        'text', 'author', 'like_count', 'video_title'
    ]].copy()

    # 'text' 컬럼의 양쪽 끝 공백 제거 (효율적인 pandas 방식)
    processed_df['text'] = processed_df['text'].str.strip()

    print(f"총 {len(processed_df)}개의 댓글 처리 완료!")
    return processed_df

def generate_and_save_ai_report(
    genai_client,
    videos_df,
    comments_df,
    meaningful_comments,
    channel_name,
    prompt,
    save_path=SAVE_PATH,
    model_name="gemini-2.0-flash"
):
    """
    영상과 댓글 데이터를 기반으로 AI 분석 리포트를 생성하고 파일로 저장합니다.

    Args:
        genai_client: Gemini AI 클라이언트 객체
        model_name (str): 사용할 Gemini 모델 이름 (예: 'gemini-1.5-flash')
        videos_df (pd.DataFrame): 분석할 영상 정보 데이터프레임
        comments_df (pd.DataFrame): 전체 댓글 데이터프레임 (통계용)
        meaningful_comments (list): AI에게 전달할 주요 댓글 리스트
        channel_name (str): 분석할 채널의 이름
        view_threshold (int): 분석 조건에 명시할 조회수 기준
        save_path (str): 분석 리포트 파일을 저장할 경로

    Returns:
        str: 저장된 파일의 경로. 실패 시 None 반환.
    """
    if videos_df.empty or comments_df.empty:
        print("영상 또는 댓글 데이터가 부족하여 AI 분석을 건너뜁니다.")
        return None

    print("\n=================================================================")
    print(" AI 분석 리포트 생성 시작")
    print("=================================================================")
    print("AI 분석용 데이터 준비 중...")

    # 영상 정보를 텍스트로 정리
    video_summary = []
    for _, video in videos_df.iterrows():
        summary = f"""제목: {video['title']}
조회수: {video['view_count']:}회
좋아요: {video['like_count']:}개
댓글: {video['comment_count']:}개
업로드: {video['upload_date']}
재생시간: {video['duration']}"""
        video_summary.append(summary)
    videos_text = '\n\n'.join(video_summary)

    # 댓글 정보를 텍스트로 정리
    comments_for_ai = []

    for _, comment in meaningful_comments.iterrows():
      # 댓글을 읽기 쉽게 정리
      comment_line = f"[{comment['like_count']} LIKES ] {comment['author']}: {comment['text']}"
      comments_for_ai.append(comment_line)
    # AI에게 분석을 요청할 프롬프트 작성
    analysis_prompt = f"""YouTube 채널 '{channel_name}' 분석 데이터입니다.
방송 PD와 콘텐츠 기획자를 위한 실용적인 분석을 해주세요.

=== 분석 대상 ===
채널: {channel_name}

=== 인기 영상 성과 ===
{videos_text}

=== 시청자 댓글 반응 (댓글 좋아요 순) ===
{comments_for_ai}

{prompt}
"""

    try:
        print(f"AI 분석 요청 중... (모델: {model_name})")
        response = genai_client.models.generate_content(
            model=model_name, # 모델 이름 형식에 맞게 수정
            contents=analysis_prompt
        )
        ai_analysis = response.text

        print("✅ AI 분석 완료!")
        print("\n" + "="*60)
        print("AI 분석 결과")
        print("="*60)
        print(ai_analysis)
        print("="*60)

        # AI 분석 결과를 파일로 저장
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        analysis_filename = f"ai_analysis_{channel_name}_{timestamp}.txt"
        analysis_path = os.path.join(save_path, analysis_filename)

        with open(analysis_path, 'w', encoding='utf-8') as f:
            f.write(f"=== {channel_name} AI 분석 리포트 ===\n")
            f.write(f"분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"분석 영상: {len(videos_df)}개\n")
            f.write(f"분석 댓글: {len(comments_df)}개\n")
            f.write(f"AI 전달 댓글: {len(meaningful_comments)}개\n\n")
            f.write("="*60 + "\n")
            f.write(ai_analysis)

        print(f"\n✅ AI 분석 결과 저장 완료: {analysis_path}")
        return analysis_path

    except Exception as e:
        print(f"❌ AI 분석 중 오류 발생: {e}")
        print("API 키를 확인하거나 나중에 다시 시도해주세요.")
        return None


def retrieve_channel_videos(channel_name,  sort_by='popular', start_date=None, end_date=None, limit=MAX_VIDEOS):
  channel_id = get_channel_id_by_handle(channel_name)
  videos = get_all_channel_videos(youtube, channel_id)
  videos = analyze_channel_videos(videos, sort_by, start_date, end_date, limit)
  videos_df = convert_dataframe_from_video_data(videos)
  return videos_df

def save_raw_data(videos_df, comments_df):
  timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
  if not videos_df.empty:
    video_filename = f"videos_{timestamp}.csv"
    video_path = os.path.join(SAVE_PATH, video_filename)
    # 한글이 깨지지 않도록 utf-8-sig 인코딩 사용
    videos_df.to_csv(video_path, index=False, encoding='utf-8-sig')
    print(f"영상 데이터 저장: {video_filename}")

  # 댓글 정보 저장 (CSV 파일로)
  if not comments_df.empty:
      comments_filename = f"comments_{timestamp}.csv"
      comments_path = os.path.join(SAVE_PATH, comments_filename)
      comments_df.to_csv(comments_path, index=False, encoding='utf-8-sig')
      print(f"댓글 데이터 저장: {comments_filename}")

  print(f"저장 완료! 파일 위치: {SAVE_PATH}")

설정 완료!
저장 위치: /content/drive/MyDrive/youtube_analysis/
YouTube API 연결 성공!
Gemini AI 연결 성공!


In [19]:
prompt="""
=== 분석 요청사항 ===
다음 관점에서 분석해주세요:
1. 현재 인기 영상의 성공 요인 (제목 패턴, 영상 길이, 콘텐츠 스타일)
2. 댓글을 통한 시청자 반응 분석 (만족도, 주요 관심사, 긍정/부정 의견)
3. 실행 가능한 콘텐츠 전략 제안 (기획 아이디어 3가지, 참여도 증진 방안)
4. 주의사항 및 개선점

방송 현업에서 바로 쓸 수 있는 구체적이고 실용적인 조언을 한국어로 해주세요.
댓글 내용을 근거로 구체적인 예시를 들어 설명해주세요.
"""

In [22]:
channel_name = "@syukaworld" #분석하고자 하는 채널 이름
sort_by = "latest" # 비디오 정렬 방법 latest, popular
start_date = None # 필터 시작 날짜, 예: 2025-09-01
end_date = None # 필터 끝 날짜, 예: 2025-09-18
num_videos_to_analyze = 10 #분석하고자 하는 비디오 개수
max_comments_per_video = 20 # 수집할 비디오 당 댓글 개수
model_name = "gemini-2.5-flash" #제미나이 모델

videos_df = retrieve_channel_videos(channel_name,sort_by=sort_by, start_date=start_date, end_date=end_date, limit=num_videos_to_analyze)
comments_df = collect_video_comments(videos_df, max_comments_per_video=max_comments_per_video)
save_raw_data(videos_df, comments_df)
meaningful_comments = preprocess_comments(comments_df)
generate_and_save_ai_report(genai_client, videos_df, comments_df, meaningful_comments, channel_name, prompt, model_name=model_name)


영상 댓글 수집 시작

[1/10] '미국을 반으로 갈라버린, 인플루언서 암살 사건...' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[2/10] '초유의 한국인 근로자 추방사태...' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[3/10] '코스피 역사상 최고점, 황스피의 시대가 왔다....' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[4/10] '위기를 맞은 '포토샵 기업', 어도비...' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[5/10] '한국은행 파격제안, '규제 풀고 택시 면허 사서 없애라'...' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[6/10] '1%가 75%의 갈등을 생산하는 시대...' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[7/10] ''노란 봉투법' 시행 확정, 무슨 법인가...' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[8/10] '반미 연합 3대장, 전면에 나선 김정은...' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[9/10] '물 위를 날아다니는 소금쟁이의 비밀...' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

[10/10] '세상에 공짜 점심은 없다....' 댓글 수집 중...
  ✅ 20개 댓글 수집 완료

댓글 수집 완료! 총 200개 댓글 수집
영상 데이터 저장: videos_20250918_012150.csv
댓글 데이터 저장: comments_20250918_012150.csv
저장 완료! 파일 위치: /content/drive/MyDrive/youtube_analysis/
총 200개의 댓글 처리 완료!

 AI 분석 리포트 생성 시작
AI 분석용 데이터 준비 중...
AI 분석 요청 중... (모델: gemini-2.5-flash)
✅ AI 분석 완료!

AI 분석 결과
'@syukaworld' 채널 분석 데이터를 바탕으로 방송 PD와 콘텐츠 기획자를 위한 실용적인 분석 및 제안을 드립니다.

---

### **1

'/content/drive/MyDrive/youtube_analysis/ai_analysis_@syukaworld_20250918_012333.txt'